In [1]:
def negative_class_edit(Y):
    Ytemp = Y
    for i,cl in enumerate(Ytemp):
        if cl == 0:
           Ytemp[i] = -1
    return Ytemp

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import mltools as ml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report,confusion_matrix,roc_auc_score
from imblearn.combine import SMOTETomek
from sklearn.linear_model import LogisticRegression
from sklearn.externals import joblib

In [3]:
#change path if your notebook is not in the same directory as your data files are
#this trains on the entire data set.
sc1 = StandardScaler()
X = np.genfromtxt("X_train.txt",delimiter=None)
Y = np.genfromtxt("Y_train.txt",delimiter=None)
train_data = X
train_Y = negative_class_edit(Y)
smotetomek = SMOTETomek(random_state = 0)
sc1.fit(train_data)
train_data = sc1.transform(train_data)
train_data,train_Y = smotetomek.fit_sample(train_data,train_Y)

In [4]:
Xtest = np.genfromtxt("X_test.txt",delimiter=None)
Xtest = sc1.transform(Xtest)

In [14]:
#change number of boosts if you want more or less learners
nboosts = 14
learner = [None] * nboosts
mTest = train_data.shape[0]
alpha = [1] * nboosts
weights = [1.0]*mTest
for i in range(len(weights)):
    weights[i] /= mTest
for i in range(nboosts):
    #print i
    learner[i] = LogisticRegression()
    learner[i].fit(train_data,train_Y,sample_weight = weights)
    Yhat = learner[i].predict(train_data)
    e = np.dot(weights,train_Y != Yhat)
    alpha[i] = .5 * np.log((1.0-e)/e)
    weights *=np.exp(-alpha[i] * train_Y * Yhat)
    weights /= weights.sum()

    
#np.savetxt("alpha.txt", alpha)
    
    
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13


In [16]:
#np.savetxt("alpha.txt", alpha)

In [17]:
mTest = Xtest.shape[0]
predict = np.zeros((mTest,))
test_probs = np.zeros((mTest,))
for i in range(nboosts):
    predict += alpha[i] * learner[i].predict(Xtest)
    
predict = np.sign(predict)


In [18]:

def predict_soft(X):
    classes = 2
    prob = np.zeros((X.shape[0],2))
    for i in range(len(learner)):
        prob += alpha[i] * learner[i].predict_proba(X)
    prob /= sum(alpha)
    prob = np.exp(prob)
    normalizer = prob.sum(axis=1)[:, np.newaxis]
    normalizer[normalizer == 0.0] = 1.0
    prob /= normalizer
    return prob

probs = predict_soft(Xtest)
    
print probs.shape
    

(200000L, 2L)


In [19]:
print predict_soft(train_data)

[[ 0.49974657  0.50025343]
 [ 0.51210432  0.48789568]
 [ 0.48647189  0.51352811]
 ..., 
 [ 0.5111964   0.4888036 ]
 [ 0.49252147  0.50747853]
 [ 0.51014152  0.48985848]]


In [15]:
Yte = np.vstack((np.arange(Xtest.shape[0]), probs[:,1])).T
print Yte
np.savetxt('Y_submit.txt', Yte, '%d, %.2f', header='ID,Prob1', comments='', delimiter=',')

[[  0.00000000e+00   4.95465404e-01]
 [  1.00000000e+00   5.02252771e-01]
 [  2.00000000e+00   4.85956608e-01]
 ..., 
 [  1.99997000e+05   4.84115349e-01]
 [  1.99998000e+05   4.87850570e-01]
 [  1.99999000e+05   4.84628841e-01]]
